In [1]:
import torch
import math
from pycircstat.descriptive import cdiff
from hipposlam.DataLoaders import ContrastiveEmbeddingDataloader


dataloader = ContrastiveEmbeddingDataloader('data/VAE/annotations2.csv', 'data/VAE/embeds2', 32, [0, 8000])



In [2]:
len(dataloader)

8000

In [3]:
for data, masks in dataloader.iterate():
    break

8000


In [24]:
labels = data[1]
labels

tensor([[ -3.2405,   1.2410,  -1.8222],
        [-15.6609,   1.8507,   2.5721],
        [ -7.1368,   8.6791,   2.2646],
        [ -2.7285,  -8.1216,  -1.5723],
        [ -0.3038,   0.8466,  -2.4969],
        [ -4.5714,   1.9548,  -0.7829],
        [ -5.7717,  15.9818,   1.5724],
        [ -0.1300,  -6.8448,  -3.1025],
        [ -0.6458,   2.4312,   1.8557],
        [ -2.8518,  -0.3105,   0.6447],
        [ -0.8175,  -4.0785,   2.4012],
        [ -9.4129,   0.7393,   1.5683],
        [  3.6546,   2.4224,  -1.8217],
        [ -1.5883,  -3.7874,  -2.5772],
        [ -2.9009,   4.0978,  -2.3104],
        [ -2.8322,   1.5540,  -2.7772],
        [ -5.0726,   3.7732,  -2.0341],
        [ -6.7394,   5.5199,   1.3050],
        [ -9.2867,   3.9579,  -1.2038],
        [  0.1815,  -1.7132,  -2.1858],
        [ -0.9093,  -0.4236,  -0.0302],
        [ -9.8570,   6.7360,  -1.8018],
        [ -3.4154,   4.8863,  -2.3703],
        [ -0.2710,   1.0543,   0.5954],
        [-15.1525,  13.6749,   0.2120],


In [27]:
torch.cdist(labels, labels).shape

torch.Size([32, 32])

In [29]:
import pandas as pd

df = pd.read_csv('data/VAE/annotations2.csv', header=0)

torch.from_numpy(df.to_numpy())

tensor([[ 0.0000e+00, -5.9450e-01, -3.4280e+00,  1.5941e-01],
        [ 1.0000e+00,  7.3239e-01, -4.6418e+00, -7.9862e-01],
        [ 2.0000e+00,  1.3416e-01, -5.0026e+00, -2.8228e+00],
        ...,
        [ 1.0029e+04, -1.5864e+01,  2.5562e-01, -2.6548e+00],
        [ 1.0030e+04, -1.5879e+01, -2.7449e-01, -1.9244e+00],
        [ 1.0031e+04, -1.5250e+01, -1.6071e+00, -1.2514e+00]],
       dtype=torch.float64)

In [30]:
data = torch.rand(32, 3) * torch.pi * 2 - torch.pi
x, y, a = data[:, 0], data[:, 1], data[:, 2]

xmin, ymin = torch.min(data[:, [0, 1]], dim=0)[0]
xmax, ymax = torch.max(data[:, [0, 1]], dim=0)[0]
xnorm = (x-xmin)/(xmax-xmin)
ynorm = (y-ymin)/(ymax-ymin)
adiff = cdiff(a.reshape(-1, 1), a.reshape(1, -1)) / torch.pi

In [31]:
posdist = torch.sqrt((xnorm.reshape(-1, 1) - xnorm.reshape(1, -1)) ** 2 + (ynorm.reshape(-1, 1) - ynorm.reshape(1, -1)) ** 2) / math.sqrt(2)
sim_mask_tmp = ((posdist < dist_thresh) & (adiff < adiff_thresh)).to(torch.uint8)
no_diag_mask = 1 - torch.eye(sim_mask_tmp.shape[0]).to(torch.uint8)
sim_mask = sim_mask_tmp & no_diag_mask
dissim_mask = 1 - sim_mask_tmp

plt.imshow(sim_mask)
plt.colorbar()

NameError: name 'dist_thresh' is not defined

In [28]:
sim_mask.dtype

NameError: name 'sim_mask' is not defined

In [32]:
torch.tensor([0, 1, 1], dtype=torch.uint8) * torch.tensor([1, 2, 3], dtype=torch.float32)

tensor([0., 2., 3.])